In [1]:
import sys
from pathlib import Path
from bayesianquilts.tf.parameter import Interactions, Decomposed


2024-12-03 15:22:52.333133: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 15:22:52.336777: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 15:22:52.346837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733239372.363468 1342401 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733239372.368264 1342401 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 15:22:52.386796: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Let's consider extending the linear regression problem

$$ y = \vec{x} \cdot \vec{\beta} $$

so that $\beta$ can vary. Let's assume that $\vec{\beta}$ is $100$ dimensional

Let's say you have some number of **discrete** factors over which you would like to vary $\beta$, for example, you might have sex, race, and whether one is currently a smoker. Let's assume you have 5 possible values for race, two for sex, and two for smoker status. Then you have $2\times 5\times 2=20$ different possible groups over which to fit your model, and $20\times 100=2000$ total regression parameters.

A common approach is to divide the data into these 20 groups, and fit 20 separate models. One can see who this procedure would be statistically problematic  already with this simple example. We would like to define a way of regularizing such a problem.

# Parameter decomposition method

In dividing the data into 20 groups, one doesn't allow the groups to share information. Instead, let's consider decomposing the parameter $\beta$ in terms of the order of interactions, so $\beta=\beta_0 + \beta_{sex}+\beta_{race}+\beta_{smoking} + \beta_{sex, gender} + \ldots $

## WHY??

The reason we  want to do this is because we can increase the strength of the regularization for higher order terms, in effect more-strongly forcing more of the higher-order contributions to zero by default. What this procedure does is partially pool the information in the dataset so that the model is effectively lower-order except in places where the data supports high-order contributions.

The `Interaction` and `Decomposed` classes work together in creating this decomposition:

In [2]:
interaction = Interactions(
    [
        ('sex', 2),
        ('race', 5),
        ('smoking', 2)
    ], exclusions=[]
    )

print(interaction)

Interaction dimensions: [('sex', 2), ('race', 5), ('smoking', 2)]


In [3]:
beta = Decomposed(
    param_shape=[100],
    interactions=interaction,
    name='beta'
)
print(beta)

Parameter shape: [100] 
Interaction dimensions: [('sex', 2), ('race', 5), ('smoking', 2)] 
Component tensors: 8 
Effective parameter cardinality: 2000 
Actual parameter cardinality: 5400



We have created a representation for $\vec{\beta}$ that consists of eight component parameters:

In [ ]:
print(beta._tensor_part_shapes)

[100]


The `Decomposed` class created the constituent parameters and initialized them to a value which is default of $\vec{0}$.

We can also choose to exclude certain interactions. Let's say we think the model should exclude the interaction between race and sex not qualified by smoking:

In [7]:
interaction = Interactions(
    [
        ('sex', 2),
        ('race', 5),
        ('smoking', 2)
    ], exclusions=[('race', 'sex')]
    )

beta = Decomposed(
    param_shape=[100],
    interactions=interaction,
    name='beta'
)
print(beta)
print(beta._tensor_part_shapes)

Parameter shape: [100] 
Interaction dimensions: [('sex', 2), ('race', 5), ('smoking', 2)] 
Component tensors: 7 
Effective parameter cardinality: 2000 
Actual parameter cardinality: 4400

{'beta__': [1, 1, 1, 100], 'beta__smoking': [1, 1, 2, 100], 'beta__race': [1, 5, 1, 100], 'beta__race_smoking': [1, 5, 2, 100], 'beta__sex': [2, 1, 1, 100], 'beta__sex_smoking': [2, 1, 2, 100], 'beta__sex_race_smoking': [2, 5, 2, 100]}


You see that the number of constituent tensors is now $7$. In practice, we might wish to exclude higher order terms in order to save memory.

Now we have created the decomposition. Let's use the decomposition. Suppose I have a sample of people:

1. sex=1, race=1, smoking=1
2. sex=0, race=3, smoking=0
3. sex=0, race=2, smoking=1
4. sex=1, race=0, smoking=1

 and I want to retrieve the effect values of $\vec{\beta}$ for these people. The `Decomposed` class takes care of this lookup

In [8]:
indices = [
    [1, 1, 1],
    [0, 3, 0],
    [0, 2, 1],
    [1, 0, 1]
]

beta_effective = beta.lookup(indices)
print(beta_effective.shape)



(4, 100)


You see that I have $4 \times 100$ values that are returned, where each row corresponds to the regression parameter vector for each of the people.

# Parameter batches

TFP works on parameters in sample batches. The `Decomposed` class handles batching. Let's generate a batch of size $5$ of the component tensors that add to $\vec{\beta}$:

In [9]:
tensors, tensor_names, tensor_shapes = beta.generate_tensors(batch_shape=[5])
print([f"{k}: {v.shape}" for k, v in tensors.items()])

['beta__: (5, 1, 100)', 'beta__smoking: (5, 2, 100)', 'beta__race: (5, 5, 100)', 'beta__race_smoking: (5, 10, 100)', 'beta__sex: (5, 2, 100)', 'beta__sex_smoking: (5, 4, 100)', 'beta__sex_race_smoking: (5, 20, 100)']


now let's query the batched parameter to get batched effective values:

In [10]:
beta.set_params(tensors)
effective_batched = beta.lookup(indices)
print(effective_batched.shape)


(4, 100)


So you see that for each of the 4 people we have a batch of size 5, of the vector $\vec{\beta}$.

# Inflating and deflating the indices

Tensorflow has an intrinisic limitation in the tensor rank allowed in its most basic operations. To get around this limitation, we can inflate and deflate the interaction index dimensions. Let's take a look at generating deflated constituent parameters and inflating them:

In [13]:
tensors, tensor_names, tensor_shapes = beta.generate_tensors(batch_shape=[5])
print([f"{k}: {v.shape}" for k, v in tensors.items()])

['beta__: (5, 1, 100)', 'beta__smoking: (5, 2, 100)', 'beta__race: (5, 5, 100)', 'beta__race_smoking: (5, 10, 100)', 'beta__sex: (5, 2, 100)', 'beta__sex_smoking: (5, 4, 100)', 'beta__sex_race_smoking: (5, 20, 100)']


So you see here that the middle axes corresponding to the interaction indices has collapsed into a single axis in each of the parameter tensors. To re-inflate, we have the method:

In [15]:
t1 = beta.inflate_indices(tensors=tensors)
print([f"{k}: {v.shape}" for k, v in t1.items()])

['beta__: (5, 1, 1, 1, 100)', 'beta__smoking: (5, 1, 1, 2, 100)', 'beta__race: (5, 1, 5, 1, 100)', 'beta__race_smoking: (5, 1, 5, 2, 100)', 'beta__sex: (5, 2, 1, 1, 100)', 'beta__sex_smoking: (5, 2, 1, 2, 100)', 'beta__sex_race_smoking: (5, 2, 5, 2, 100)']


And all is good in the world. Happy 2022!